<a href="https://colab.research.google.com/github/Leonardojdss/MedGemma-analyze-image/blob/main/MedGemma_analyze_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# pip install accelerate
from transformers import AutoProcessor, AutoModelForImageTextToText
from PIL import Image
import requests
import torch

model_id = "google/medgemma-4b-it"

model = AutoModelForImageTextToText.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
processor = AutoProcessor.from_pretrained(model_id)

# Image attribution: Stillwaterising, CC0, via Wikimedia Commons
image_url = "https://upload.wikimedia.org/wikipedia/commons/c/c8/Chest_Xray_PA_3-8-2010.png"
image = Image.open(requests.get(image_url, headers={"User-Agent": "example"}, stream=True).raw)

messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "Você é um especialista em radiologia."}]
    },
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "descreva o raio X-ray"},
            {"type": "image", "image": image}
        ]
    }
]

inputs = processor.apply_chat_template(
    messages, add_generation_prompt=True, tokenize=True,
    return_dict=True, return_tensors="pt"
).to(model.device, dtype=torch.bfloat16)

input_len = inputs["input_ids"].shape[-1]

with torch.inference_mode():
    generation = model.generate(**inputs, max_new_tokens=200, do_sample=False)
    generation = generation[0][input_len:]

decoded = processor.decode(generation, skip_special_tokens=True)
print(decoded)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Com prazer! Analisando a imagem fornecida, podemos identificar os seguintes elementos e características:

**Imagem:**

*   **Tipo:** Radiografia de tórax (ou radiografia de peito).
*   **Ângulo:** A imagem é tirada em perfil, ou seja, do lado.
*   **Foco:** O foco principal é o tórax, incluindo os pulmões, coração, grandes vasos, costelas, esterno e coluna vertebral.

**Características:**

*   **Pulmões:** Os pulmões aparecem como áreas escuras (opacas) devido à sua composição gasosa. Podemos observar a estrutura pulmonar, como os vasos sanguíneos e as pequenas estruturas que compõem os pulmões.
*   **Coração:** O coração está localizado no centro do tórax, entre os pulmões. Sua silhueta é visível, mas a imagem não é ideal para avaliar a sua estrutura interna em detalhes.

